
# *Data Analyst - Assignment*

In [6]:
import math
import numpy as np
import pandas as pd

# File paths
files = [
    "d:/Projects/COINTAB/Assignment_Data/Company X - Order Report.xlsx",
    "d:/Projects/COINTAB/Assignment_Data/Company X - Pincode Zones.xlsx",
    "d:/Projects/COINTAB/Assignment_Data/Company X - SKU Master.xlsx",
    "d:/Projects/COINTAB/Assignment_Data/Courier Company - Invoice.xlsx",
    "d:/Projects/COINTAB/Assignment_Data/Courier Company - Rates.xlsx"
]

# Read data from Excel files
order_report, pin_zone, sku, invoice, rates = [pd.read_excel(file) for file in files]

# Check for missing values
missing_values = {f"{file}": df.isna().sum() for file, df in zip(files, [order_report, pin_zone, sku, invoice, rates])}
print("\n".join([f"Missing Values in {file}:\n{values}" for file, values in missing_values.items()]))

# Check for duplicate rows
duplicate_values = {f"{file}": df.duplicated().sum() for file, df in zip(files, [order_report, pin_zone, sku, invoice, rates])}
print("\n".join([f"Number of duplicate rows in {file}: {value}" for file, value in duplicate_values.items()]))

# Remove duplicate rows
order_report, sku, pin_zone = [df.drop_duplicates() for df in [order_report, sku, pin_zone]]

# Merge relevant data
data = order_report.merge(sku, how='left', on='SKU')
data['Total_Weight_by_X(kg)'] = data['Weight (g)'] * data['Order Qty'] / 1000
data = data.drop(labels=['Order Qty', 'Weight (g)', 'SKU'], axis=1)
data = data.groupby(by=['ExternOrderNo'], as_index=False).sum()
data = data.merge(invoice, how='left', left_on='ExternOrderNo', right_on='Order ID')
data = data.merge(pin_zone, how='left', on=['Warehouse Pincode', 'Customer Pincode']).rename(columns={'Zone_x': 'Zone_by_Courier', 'Zone_y': 'Zone_by_X'})

rates['Zone'] = rates['Zone'].str.lower()
data = data.merge(rates, how='left', left_on='Zone_by_X', right_on='Zone')

# Calculate charges
data['Forward_additional_charge'] = (data['Total_Weight_by_X(kg)'] / data['Weight Slabs']).astype(int) * data['Forward Additional Weight Slab Charge']
data['RTO_fixed_charge'] = data.apply(lambda row: row['RTO Fixed Charge'] if row['Type of Shipment'] == 'Forward and RTO charges' else 0, axis=1)
data['RTO_additional_charge'] = data.apply(lambda row: math.floor(row['Total_Weight_by_X(kg)'] / row['Weight Slabs']) * row['RTO Additional Weight Slab Charge'] if row['Type of Shipment'] == 'Forward and RTO charges' else 0, axis=1)
data['Expected_charges'] = data[['Forward Fixed Charge', 'Forward_additional_charge', 'RTO_fixed_charge', 'RTO_additional_charge']].sum(axis=1)
data['diff_charges'] = data['Expected_charges'] - data['Billing Amount (Rs.)']
data['Weight_slabs_Courier'] = data['Zone_by_Courier'].map({'a': 0.25, 'b': 0.5, 'c': 0.75, 'd': 1.25, 'e': 1.5})
data['over_or_under_charged'] = np.select([data['diff_charges'] == 0, data['diff_charges'] > 0, data['diff_charges'] < 0],
                                           ['X correctly charged', 'X Undercharged', 'X Overcharged'])

# Prepare output DataFrames
output = data[['Order ID', 'AWB Code', 'Total_Weight_by_X(kg)', 'Weight Slabs', 'Charged Weight', 'Weight_slabs_Courier', 'Zone_by_X',
               'Zone_by_Courier', 'Expected_charges', 'Billing Amount (Rs.)', 'diff_charges']]

summary = pd.DataFrame(index=['Total orders where X has been correctly charged',
                              'Total Orders where X has been overcharged',
                              'Total Orders where X has been undercharged'], columns=['Count', 'Amount(Rs.)'])
summary['Count'] = [(data['diff_charges'] == 0).sum(),
                    (data['diff_charges'] < 0).sum(),
                    (data['diff_charges'] > 0).sum()]
summary['Amount(Rs.)'] = [data.loc[data['diff_charges'] == 0, 'Billing Amount (Rs.)'].sum(),
                          abs(data.loc[data['diff_charges'] < 0, 'diff_charges'].sum()),
                          data.loc[data['diff_charges'] > 0, 'diff_charges'].sum()]

# Write output to Excel files
output.to_excel("d:/Projects/COINTAB/Assignment_Data/Output.xlsx", index=False)
summary.to_excel("d:/Projects/COINTAB/Assignment_Data/Summary.xlsx")


Missing Values in d:/Projects/COINTAB/Assignment_Data/Company X - Order Report.xlsx:
ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64
Missing Values in d:/Projects/COINTAB/Assignment_Data/Company X - Pincode Zones.xlsx:
Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64
Missing Values in d:/Projects/COINTAB/Assignment_Data/Company X - SKU Master.xlsx:
SKU           0
Weight (g)    0
dtype: int64
Missing Values in d:/Projects/COINTAB/Assignment_Data/Courier Company - Invoice.xlsx:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64
Missing Values in d:/Projects/COINTAB/Assignment_Data/Courier Company - Rates.xlsx:
Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Sla